Preprocesamiento de los artículos

In [8]:
import os
import pandas as pd

OUTPUT_FILE = "/home/jennifer/Descargas/papers_guardados-2.csv"

if os.path.exists(OUTPUT_FILE):
    df_existente = pd.read_csv(OUTPUT_FILE)

    df_limpio = df_existente[df_existente['contenido'].notna() & (df_existente['contenido'] != '')]
    df_limpio.to_csv(OUTPUT_FILE, index=False)

    print(f"Archivo limpiado guardado.: {len(df_limpio)}")
    print(f"Archivo limpiado guardado.: {len(df_existente)}")


Archivo limpiado guardado.: 865
Archivo limpiado guardado.: 865


In [9]:
import pandas as pd

df = pd.read_csv('/home/jennifer/Documentos/tercer_año/segundo_semestre/SRI/proyecto final/papers_guardados.csv')  
df1=pd.read_csv("/home/jennifer/Descargas/papers_guardados-2.csv")  

# Concatenar df y df1 fila a fila 
df = pd.concat([df, df1], axis=0, ignore_index=True)
print(df.columns,len(df))

Index(['title', 'doi', 'landing_page', 'authors', 'published', 'abstract',
       'language', 'pdf_url', 'contenido'],
      dtype='object') 2865


In [10]:
import pandas as pd
import re
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

def normalize_text(text):
    """ 
    Normaliza un texto eliminando caracteres especiales y estandarizando formatos.
    Args:
        text(str): Texto sin procesar
    Return:
        str: Texto preprocesado para caracteres especiales.
    """
    if type(text)==float:
        print(text)
    text = text.lower()
    text = re.sub(r"[^a-záéíóúüñ\s]", "", text)
    text = text.replace('“', '"').replace('”', '"')
    text = text.replace("’", "'").replace("–", "-").replace("—", "-")
    text = re.sub(r'\s+', ' ', text)
    return text
urls=[]
count=0
def remove_sections(text):
    """    
    Elimina secciones específicas del contenido del artículo como Referencias y Agradecimientos, así como todo lo que contenga después 
    Args:
        text(str): Contenido del artículo para eliminar secciones
    Return:    
        str: El texto anterior eliminando las secciones anteriores
    """
    global count
    count+=1
    after_refs = re.split(r'\n(references|bibliography|acknowledg(e)?ments)\b', text, flags=re.IGNORECASE)  

    return after_refs[0]


def clean_text(text):
    """
    Realiza limpieza del texto eliminando caracteres invisibles y espacios múltiples.
    Args:
        text (str): Texto a limpiar.
    Return:    
        str: Texto limpio con espaciado uniforme

    """
   
    if not isinstance(text, str): 
        return ""
    text = re.sub(r'[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F]', ' ', text)
    text = re.sub(r'[ \t]+', ' ', text)
    return text.strip()

df['clean_abstract'] = df['abstract'].apply(clean_text)
df['clean_title'] = df['title'].apply(clean_text)
df['clean_contenido']=df['contenido'].apply(remove_sections)
df['clean_contenido'] = df['clean_contenido'].apply(normalize_text)
df['clean_contenido'] = df['clean_contenido'].apply(clean_text)
df = df[df['clean_contenido'].str.len() > 200]  


Combirtiendo a embedings los artículos

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import time

print(f"[{time.strftime('%H:%M:%S')}] INFO: Convirtiendo DataFrame a Documentos LangChain...")
documents = []
for _, row in df.iterrows():
    content = f"{row['clean_abstract']}\n\n{row['clean_contenido']}"
    
    metadata = {
        'titulo': row['clean_title'], 
        'doi': row['doi'],
        'autores': row['authors'],
        'publicado': row['published'],
        'idioma': row['language'],
        'url_pdf': row['pdf_url'],
        'url_landing': row['landing_page'],
    }
    
    documents.append(Document(page_content=content, metadata=metadata))
print(f"[{time.strftime('%H:%M:%S')}] INFO: {len(documents)} documentos creados.")


print(f"[{time.strftime('%H:%M:%S')}] INFO: Iniciando división de documentos en chunks...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, 
    chunk_overlap=200, 
    separators=["\n\n", "\n", r"(?<=\. )", " "],  # Respetar estructura
    length_function=len,
    is_separator_regex=True
)

chunks = []
for i, doc in enumerate(documents):
    splits = text_splitter.split_documents([doc])
    for j, split in enumerate(splits):
        split.metadata['fragmento_id'] = f"{doc.metadata['doi']}_part{j+1}"
    chunks.extend(splits)
    print(f"[{time.strftime('%H:%M:%S')}] DEBUG: Documento {i+1} dividido en {len(splits)} fragmentos.")

print(f"[{time.strftime('%H:%M:%S')}] INFO: Total de {len(chunks)} fragmentos creados.")

print(f"[{time.strftime('%H:%M:%S')}] INFO: Cargando modelo de embeddings HuggingFace...")
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", encode_kwargs={'normalize_embeddings': True}
)
print(f"[{time.strftime('%H:%M:%S')}] INFO: Modelo de embeddings cargado correctamente.")


print(f"[{time.strftime('%H:%M:%S')}] INFO: Creando base vectorial con ChromaDB...")
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embed_model,
    persist_directory="./repositorio_vectorial",
    collection_metadata={"hnsw:space": "cosine"}  # Optimizado para similitud
)

print(f"[{time.strftime('%H:%M:%S')}] INFO: Guardado base vectorial en disco...")
#vector_db.persist()


[19:19:45] INFO: Convirtiendo DataFrame a Documentos LangChain...
[19:19:45] INFO: 2835 documentos creados.
[19:19:45] INFO: Iniciando división de documentos en chunks...
[19:19:45] DEBUG: Documento 1 dividido en 17 fragmentos.
[19:19:45] DEBUG: Documento 2 dividido en 22 fragmentos.
[19:19:45] DEBUG: Documento 3 dividido en 22 fragmentos.
[19:19:45] DEBUG: Documento 4 dividido en 4 fragmentos.
[19:19:45] DEBUG: Documento 5 dividido en 6 fragmentos.
[19:19:45] DEBUG: Documento 6 dividido en 15 fragmentos.
[19:19:45] DEBUG: Documento 7 dividido en 33 fragmentos.
[19:19:45] DEBUG: Documento 8 dividido en 19 fragmentos.
[19:19:45] DEBUG: Documento 9 dividido en 16 fragmentos.
[19:19:45] DEBUG: Documento 10 dividido en 23 fragmentos.
[19:19:45] DEBUG: Documento 11 dividido en 21 fragmentos.
[19:19:45] DEBUG: Documento 12 dividido en 25 fragmentos.
[19:19:45] DEBUG: Documento 13 dividido en 15 fragmentos.
[19:19:45] DEBUG: Documento 14 dividido en 20 fragmentos.
[19:19:45] DEBUG: Documento 

AttributeError: 'Chroma' object has no attribute 'persist'

Ejemplo de recuperación de información en el repositorio vectorial

In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Cargar el repositorio existente
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = Chroma(
    persist_directory="./repositorio_vectorial",
    embedding_function=embed_model
)

# Método 1: Obtener conteo de documentos
num_docs = vector_db._collection.count()
print(f"📊 Número total de fragmentos/documentos en la base: {num_docs}")

# Método 2: Obtener algunos documentos de ejemplo
sample_docs = vector_db._collection.get(limit=5)
print("\n🔍 Muestra de documentos almacenados:")
for i, (content, metadata) in enumerate(zip(sample_docs['documents'], sample_docs['metadatas'])):
    print(f"\nDocumento {i+1}:")
    print(f"ID: {sample_docs['ids'][i]}")
    print(f"Título: {metadata.get('titulo', 'N/A')}")
    print(f"Fragmento ID: {metadata.get('fragmento_id', 'N/A')}")
    print(f"Contenido (inicio): {content[:200]}...")

📊 Número total de fragmentos/documentos en la base: 91717

🔍 Muestra de documentos almacenados:

Documento 1:
ID: 3d37af8d-5c5d-4f1b-9e79-48b50a0d7c8e
Título: Machine Learning Algorithm in Agricultural Machine Vision System
Fragmento ID: 10.38007/ml.2020.010404_part1
Contenido (inicio): scholar publishing group machine learn ing theory and practice httpsdoiorg ml issn v ol issue copyright by the authors this is an open access article distributed under the creative commons attribution...

Documento 2:
ID: 4d65ee27-83a2-4411-a5e3-af4a2d104bd4
Título: Machine Learning Algorithm in Agricultural Machine Vision System
Fragmento ID: 10.38007/ml.2020.010404_part2
Contenido (inicio): key technologies and machine learning algorithms of agricultural machine vision system and briefly discusses the syste m hardware selection and software development environment for the application of ...

Documento 3:
ID: 96a79488-92b8-41b5-8fb6-52128c9efb25
Título: Machine Learning Algorithm in Agricultural Machin